# Assignment 9 - submitted by Tarang Ranpara (202011057)

In [ ]:
! pip install sumy

     |████████████████████████████████| 92kB 4.6MB/s 
     |████████████████████████████████| 10.1MB 11.1MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21680 sha256=da1572b2fc994c3db6590e116ef5f168df2f1f5090eab77ef5863fc550d67af8
  Stored in directory: /root/.cache/pip/wheels/5a/4d/a1/510b12c5e65e0b2b3ce539b2af66da0fc57571e528924f4a52
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746863 sha256=2ec58ed5afd03f609185cc897806b109d0a54c59aa3e2f2e41959f149ff1b216
  Stored in directory: /root/.cache/pip/wheels/33/4e/a6/be297e6b83567e537bed9df4a93f8590ec01c1acfbcd405348
Successfully built breadability pycountry


In [ ]:
# importing essential libs
import os
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.evaluation.rouge import rouge_1, rouge_2
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip './drive/MyDrive/IRLAB/A10/BBC Business News.zip' -d ./data

Archive:  ./drive/MyDrive/IRLAB/A10/BBC Business News.zip
   creating: ./data/BBC Business News/News Articles/
   creating: ./data/BBC Business News/News Articles/business/
  inflating: ./data/BBC Business News/News Articles/business/001.txt  
  inflating: ./data/BBC Business News/News Articles/business/002.txt  
  inflating: ./data/BBC Business News/News Articles/business/003.txt  
  inflating: ./data/BBC Business News/News Articles/business/004.txt  
  inflating: ./data/BBC Business News/News Articles/business/005.txt  
  inflating: ./data/BBC Business News/News Articles/business/006.txt  
  inflating: ./data/BBC Business News/News Articles/business/007.txt  
  inflating: ./data/BBC Business News/News Articles/business/008.txt  
  inflating: ./data/BBC Business News/News Articles/business/009.txt  
  inflating: ./data/BBC Business News/News Articles/business/010.txt  
  inflating: ./data/BBC Business News/News Articles/business/011.txt  
  inflating: ./data/BBC Business News/News Art

In [ ]:
# Initialize paths and configurations
dataset_path = os.path.join("./data/BBC Business News/","News Articles","business")
summaries_path = os.path.join("./data/BBC Business News/","Summaries","business")

In [ ]:
language = "english"
sentence_counts = (10, 15, 20, 25)

In [ ]:
'''
  Implementing LexRank and TextRank summarization techniques for the dataset
'''

def auto_textsummarize_dataset(dataset_path, language, sentence_count, text_or_lex='text'):
    
    # Initialize dictionary which will store the generated summaries for all the documents in the path
    summary_sentences_list = []
    
    # Initialize summarizer
    stemmer = Stemmer(language)
    summarizer = TextRankSummarizer(stemmer) if text_or_lex == 'text' else LexRankSummarizer(stemmer)
    summarizer.stop_words = get_stop_words(language)
    
    doc_paths = [os.path.join(dp, fname) for dp, df, filepaths in os.walk(dataset_path) for fname in filepaths]
    
    for doc_path in doc_paths:

        # Parse and tokenize the text in the file
        doc_parser = PlaintextParser.from_file(doc_path, Tokenizer(language))
        
        # Create a list of sentences from the generated summary using the given sentence count
        summary_sentences_list.append([sentence for sentence in summarizer(doc_parser.document, sentence_count)])

    return summary_sentences_list

In [ ]:
# Evaluation function
def evaluate_rouge_1_2(dataset_path, ref_summaries_path, language, sentence_count, **kwargs):
    summ_sent_list = auto_textsummarize_dataset(dataset_path, language, sentence_count, **kwargs)
    
    avg_rouge_1_score = []
    avg_rouge_2_score = []
    
    ref_summ_paths = [os.path.join(dp, fname) 
                      for dp, df, filepaths in os.walk(ref_summaries_path) 
                      for fname in filepaths]
    
    for ref_summ_path, summ_sent in zip(ref_summ_paths, summ_sent_list):
        ref_summ_parser = PlaintextParser.from_file(ref_summ_path, Tokenizer(language))
        
        avg_rouge_1_score.append(rouge_1(summ_sent, ref_summ_parser.document.sentences))
        avg_rouge_2_score.append(rouge_2(summ_sent, ref_summ_parser.document.sentences))
    
    avg_rouge_1_score = sum(avg_rouge_1_score) / len(avg_rouge_1_score)
    avg_rouge_2_score = sum(avg_rouge_2_score) / len(avg_rouge_2_score)
    
    print(f"For sentence count {sentence_count}:\n")
    print(f"Average Rouge 1 Score: {avg_rouge_1_score}")
    print(f"Average Rouge 2 Score: {avg_rouge_2_score}")
    print("\n")

## Generating summaries and getting Rouge scores
- Summaries are generated from the dataset using TextRank and LexRank summarizers
- Different sentence counts are taken and their average rouge 1 and rouge 2 scores are calculated

In [ ]:
print("TextRank Summarizer:\n")
for sentence_count in sentence_counts:
    evaluate_rouge_1_2(dataset_path, summaries_path, language, sentence_count, text_or_lex='text')

TextRank Summarizer:

For sentence count 10:

Average Rouge 1 Score: 0.9362560146531602
Average Rouge 2 Score: 0.8757140464934037


For sentence count 15:

Average Rouge 1 Score: 0.9851875534262475
Average Rouge 2 Score: 0.9386486886310059


For sentence count 20:

Average Rouge 1 Score: 0.9949702140951422
Average Rouge 2 Score: 0.9512299413126836


For sentence count 25:

Average Rouge 1 Score: 0.9984427176241695
Average Rouge 2 Score: 0.9559376020192208




In [ ]:
print("LexRank Summarizer:\n")
for sentence_count in sentence_counts:
    evaluate_rouge_1_2(dataset_path, summaries_path, language, sentence_count, text_or_lex='lex')

LexRank Summarizer:

For sentence count 10:

Average Rouge 1 Score: 0.8056354257569128
Average Rouge 2 Score: 0.7010785291412631


For sentence count 15:

Average Rouge 1 Score: 0.933400042627417
Average Rouge 2 Score: 0.8671580805844253


For sentence count 20:

Average Rouge 1 Score: 0.9768186230431035
Average Rouge 2 Score: 0.9264571193849707


For sentence count 25:

Average Rouge 1 Score: 0.9920993056726062
Average Rouge 2 Score: 0.9472817430152636


